In [3]:
using FactCheck, ForwardDiff, Plots

In [4]:
function newton(f, x; tol_f = 1e-6, tol_x = 1e-3,
        maxiter = 1000, maxtime = 60)
    tol_g = tol_f^2
    g(x) = ForwardDiff.derivative(f, x)
    escala = max( abs(g(x)), abs(f(x)) )
    if abs(g(x)) < tol_g * escala
        return x, -1, f(x), g(x), 0, 0.0
    end
    d = -f(x)/g(x)
    iter = 0
    ef = 0
    start_time = time()
    el_time = 0.0
    while abs(f(x)) > tol_f || abs(d) > tol_x
        x = x + d
        if abs(g(x)) < tol_g * escala
            ef = -1
            break
        end
        d = -f(x)/g(x)
        iter += 1
        el_time = time() - start_time
        if iter >= maxiter
            ef = 1
            break
        end
        if el_time >= maxtime
            ef = 2
            break
        end
    end
    return x, ef, f(x), g(x), iter, el_time
end

newton (generic function with 1 method)

In [5]:
context("Exemplos básicos") do
    facts("f(x) = x^2 - 1") do
        f(x) = x^2-1
        x, ef, fx, fdx, iter, el_time = newton(f, 1.0)
        @fact x --> roughly(1.0, 1e-6)
        @fact ef --> 0
        @fact fx --> roughly(0.0, 1e-6)
        @fact iter --> less_than(1000)
        @fact el_time --> less_than(60)
    end
    
    facts("f(x) = x^2") do
        f(x) = x^2
        x, ef, fx, fdx, iter, el_time = newton(f, 1.0)
        @fact x --> roughly(0.0, 1e-3)
        @fact ef --> 0
        @fact fx --> roughly(0.0, 1e-6)
        @fact fdx --> roughly(0.0, 2e-3)
        @fact iter --> less_than(1000)
        @fact el_time --> less_than(60)
    end
end

context("Não quebre") do
    facts("f'(x) = 0") do
        f(x) = x^2 - 1
        x0 = 0.0
        x, ef, fx, fdx, iter, el_time = newton(f, x0)
        @fact ef --> -1 # -1 -> derivada zero
    end
    facts("Escala") do
        for p = [1e8; 1e-8]
            f(x) = p * (x^2 - 4)
            x, ef, fx, fdx, iter, el_time = newton(f, 1)
            @fact x --> roughly(2.0, 1e-3)
            @fact fx --> roughly(0.0, 1e-6)
            @fact ef --> 0
        end
    end
    facts("Longe da solução") do
        f(x) = x^2/(1+x^2)
        x0 = 2.0
        x, ef, fx, fdx, iter, el_time = newton(f, x0)
        @fact abs(x) --> greater_than(100)
        @fact ef --> not(0)
    end
end

context("Sem loop infinito") do
    f(x) = x^4
    x0 = 1.0
    facts("Iterações") do
        x, ef, fx, fdx, iter, el_time = newton(f, x0, maxiter=1)
        @fact ef --> not(0)
        @fact iter --> 1
    end
    
    facts("Tempo") do
        x, ef, fx, fdx, iter, el_time = newton(f, x0, maxtime=0.0)
        @fact ef --> not(0)
        @fact el_time --> greater_than(0.0)
    end
end

  > Exemplos básicos
f(x) = x^2 - 1
5 facts verified.
f(x) = x^2
6 facts verified.
  > Não quebre
f'(x) = 0
1 fact verified.
Escala
6 facts verified.
Longe da solução
2 facts verified.
  > Sem loop infinito
Iterações
2 facts verified.
Tempo
2 facts verified.


delayed_handler (generic function with 4 methods)

In [8]:
function secante(f, x; tol_f = 1e-6, tol_x = 1e-3,
        maxiter = 1000, maxtime = 60)
    d = 1
    iter = 0
    ef = 0
    start_time = time()
    el_time = 0.0
    while abs(f(x)) > tol_f || abs(d) > tol_x
        x1 = x + d
        m = (f(x1) - f(x))/(x1 - x)
        d = -f(x1)/m
        x = x1
        iter += 1
        el_time = time() - start_time
        if iter >= maxiter
            ef = 1
            break
        end
        if el_time >= maxtime
            ef = 2
            break
        end
    end
    return x, ef, f(x), iter, el_time
end

secante (generic function with 1 method)

In [9]:
x, ef, fx, iter, el_time = secante(x->x^2 - 2, 1)

(1.4142135620573204,0,-8.931455575122982e-10,6,0.002933979034423828)

In [16]:
context("Exemplos básicos") do
    facts("f(x) = x^2 - 1") do
        f(x) = x^2-1
        x, ef, fx, iter, el_time = secante(f, 1.0)
        @fact x --> roughly(1.0, 1e-6)
        @fact ef --> 0
        @fact fx --> roughly(0.0, 1e-6)
        @fact iter --> less_than(1000)
        @fact el_time --> less_than(60)
    end
    
    facts("f(x) = x^2") do
        f(x) = x^2
        x, ef, fx, iter, el_time = secante(f, 1.0)
        @fact x --> roughly(0.0, 1e-3)
        @fact ef --> 0
        @fact fx --> roughly(0.0, 1e-6)
        @fact iter --> less_than(1000)
        @fact el_time --> less_than(60)
    end
end

context("Não quebre") do
    facts("f'(x) = 0") do
        f(x) = x^2 - 1
        x0 = 0.0
        x, ef, fx, iter, el_time = secante(f, x0)
        @fact x --> roughly(1.0, 1e-6)
        @fact ef --> 0
        @fact fx --> roughly(0.0, 1e-6)
        @fact iter --> less_than(1000)
        @fact el_time --> less_than(60)    end
    facts("Escala") do
        for p = [1e8; 1e-8]
            f(x) = p * (x^2 - 4)
            x, ef, fx, iter, el_time = secante(f, 1)
            @fact x --> roughly(2.0, 1e-3)
            @fact fx --> roughly(0.0, 1e-6)
            @fact ef --> 0
        end
    end
end

context("Sem loop infinito") do
    f(x) = x^4
    x0 = 1.0
    facts("Iterações") do
        x, ef, fx, iter, el_time = secante(f, x0, maxiter=1)
        @fact ef --> not(0)
        @fact iter --> 1
    end
    
    facts("Tempo") do
        x, ef, fx, iter, el_time = secante(f, x0, maxtime=0.0)
        @fact ef --> not(0)
        @fact el_time --> greater_than(0.0)
    end
end

  > Exemplos básicos
f(x) = x^2 - 1
5 facts verified.
f(x) = x^2
5 facts verified.
  > Não quebre
f'(x) = 0
5 facts verified.
Escala
6 facts verified.
  > Sem loop infinito
Iterações
2 facts verified.
Tempo
2 facts verified.


delayed_handler (generic function with 4 methods)